<a href="https://colab.research.google.com/github/Jorya777/Jorya777/blob/main/final_assignment_CF_evaluation_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>